In [2]:
# Install required packages (uncomment if running for the first time)
!pip install -q langchain huggingface_hub transformers torch sentencepiece pypdf docarray

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.2/270.2 kB 6.7 MB/s eta 0:00:00:00:01


In [3]:
pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.7/412.7 kB 20.7 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFaceHub
from langchain.prompts import PromptTemplate


In [6]:
loader = PyPDFLoader("/kaggle/input/egjbbjb/Climate Change.pdf")  
pages = loader.load()

In [7]:
pages

[Document(metadata={'source': '/kaggle/input/egjbbjb/Climate Change.pdf', 'page': 0, 'page_label': '1'}, page_content="Climate Change \nClimate change is one of the most significant and pressing issues of our time, \naffecting every aspect of our lives, from the environment to the economy, \nhealth, and security. It refers to the long-term alteration of temperature and \ntypical weather patterns in a place. While climate change is a natural \nphenomenon, human activities, particularly since the Industrial Revolution, \nhave significantly accelerated its pace, leading to unprecedented and \npotentially catastrophic changes in the Earth's climate system. \n \nThe scientific consensus on climate change is clear: the Earth's climate is \nwarming, and human activities are the primary driver. The Intergovernmental \nPanel on Climate Change (IPCC), a leading authority on climate science, has \nreported that global temperatures have risen by approximately 1.2 degrees \nCelsius (2.2 degrees Fah

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(pages)

In [9]:
docs

[Document(metadata={'source': '/kaggle/input/egjbbjb/Climate Change.pdf', 'page': 0, 'page_label': '1'}, page_content="Climate Change \nClimate change is one of the most significant and pressing issues of our time, \naffecting every aspect of our lives, from the environment to the economy, \nhealth, and security. It refers to the long-term alteration of temperature and \ntypical weather patterns in a place. While climate change is a natural \nphenomenon, human activities, particularly since the Industrial Revolution, \nhave significantly accelerated its pace, leading to unprecedented and \npotentially catastrophic changes in the Earth's climate system. \n \nThe scientific consensus on climate change is clear: the Earth's climate is \nwarming, and human activities are the primary driver. The Intergovernmental \nPanel on Climate Change (IPCC), a leading authority on climate science, has \nreported that global temperatures have risen by approximately 1.2 degrees \nCelsius (2.2 degrees Fah

In [10]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

<ipython-input-10-3a8dd36e4865>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-mpnet-base-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [12]:
vectorstore = DocArrayInMemorySearch.from_documents(docs, embeddings)

/usr/local/lib/python3.10/dist-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')
INFO - docarray - DB config created
INFO - docarray - Runtime config created
INFO - docarray - No docs or index file provided. Initializing empty InMemoryExactNNIndex.


In [15]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

In [16]:
template = """Answer the question based on the context below. 
Provide a **detailed and well-explained** response. If you are unsure, say "I don't know".

Context: {context}

Question: {question}

Provide a **thorough** answer with examples, statistics, and insights:

Answer:"""
prompt = PromptTemplate.from_template(template)


In [17]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "YourApitoken"

In [18]:
llm = HuggingFaceHub(
    repo_id="google/flan-t5-large",  # flan-t5-xxl or Llama-2
    model_kwargs={"temperature": 0.2, "max_length": 1500}  
)

<ipython-input-18-feb6792c22c7>:1: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llm = HuggingFaceHub(


In [19]:
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])  # Combine retrieved chunks

In [20]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)

In [21]:
print("🔥 Climate Change Q&A System (Type 'exit' to quit) 🔥\n")

🔥 Climate Change Q&A System (Type 'exit' to quit) 🔥



In [22]:
while True:
    question = input("You: ")
    
    if question.lower().strip() == 'exit':
        print("👋 Exiting... Have a great day!")
        break
        
    if not question.strip():
        print("⚠️ Please enter a valid question.\n")
        continue
    result = qa_chain({"query": question})
    print("\n🤖 Bot:")
    print(f"📜 **Answer:**\n{result['result']}\n")
    
    print("📚 **Sources:**")
    for doc in result['source_documents'][:3]: 
        print(f"- {doc.metadata['source']} (Page {doc.metadata['page']+1})") 
    
    print("\n" + "—" * 60 + "\n")

You:  what is climate change?


<ipython-input-22-e9f2c6b1f242>:11: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": question})



🤖 Bot:
📜 **Answer:**
It refers to the long-term alteration of temperature and typical weather patterns in a place.

📚 **Sources:**
- /kaggle/input/egjbbjb/Climate Change.pdf (Page 1)
- /kaggle/input/egjbbjb/Climate Change.pdf (Page 2)
- /kaggle/input/egjbbjb/Climate Change.pdf (Page 3)

————————————————————————————————————————————————————————————



You:  how much temepreature is rised since 19th century?



🤖 Bot:
📜 **Answer:**
Global temperatures have risen by approximately 1.2 degrees Celsius (2.2 degrees Fahrenheit) since the late 19th century.

📚 **Sources:**
- /kaggle/input/egjbbjb/Climate Change.pdf (Page 1)
- /kaggle/input/egjbbjb/Climate Change.pdf (Page 2)
- /kaggle/input/egjbbjb/Climate Change.pdf (Page 3)

————————————————————————————————————————————————————————————



You:  exit


👋 Exiting... Have a great day!


Gujarati 

In [6]:
import os
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [7]:
pdf_path = "/kaggle/input/fhcbahbi/Climate Change.en.gu.pdf"
loader = PyPDFLoader(pdf_path)
pages = loader.load()

In [8]:
pages

[Document(metadata={'source': '/kaggle/input/fhcbahbi/Climate Change.en.gu.pdf', 'page': 0, 'page_label': '1'}, page_content='ક્લાઈમેટ ચેન્જ\nઆબોહવા પિરવર્તન એ આપણા સમયના સૌથી મહત્વપૂર્ણ અને દબાણયુક્ત મુદ્દાઓ પૈકી \nએક છે, જે આપણા જીવનના દરેક પાસાઓને અસર કરે છે, પર્યાવરણથી લઈને અર્થતંત્ર, \nઆરોગ્ય અને સુરક્ષા સુધી. તે તાપમાનના લાંબા ગાળાના ફેરફાર અને સ્થાનના લાક્ષિણક \nહવામાન પેટર્નનો સંદર્ભ આપે છે. જ્યારે આબોહવા પિરવર્તન એ કુદરતી ઘટના છે, ત્યારે \nમાનવીય પ્રવૃત્િતઓ, ખાસ કરીને ઔદ્યોિગક ક્રાંિત પછીથી, તેની ગિતમાં નોંધપાત્ર વધારો \nથયો છે, જે પૃથ્વીની આબોહવા પ્રણાલીમાં અભૂતપૂર્વ અને સંભિવત િવનાશક ફેરફારો તરફ \nદોરી જાય છે.\nઆબોહવા પિરવર્તન પર વૈજ્ઞાિનક સર્વસંમિત સ્પષ્ટ છે: પૃથ્વીની આબોહવા ગરમ થઈ રહી \nછે, અને માનવ પ્રવૃત્િતઓ પ્રાથિમક ડ્રાઈવર છે. આબોહવા િવજ્ઞાન પર અગ્રણી ઓથોિરટી \nઓન ક્લાઈમેટ ચેન્જ (IPCC) પરની આંતર સરકારી પેનલે અહેવાલ આપ્યો છે કે 19મી સદીના \nઅંતથી વૈશ્િવક તાપમાનમાં આશરે 1.2 િડગ્રી સેલ્િસયસ (2.2 િડગ્રી ફેરનહીટ) વધારો થયો છે. \nઆ વોર્િમંગ વલણ મોટે ભાગે વાતાવરણમાં ગ્રીનહાઉસ

In [10]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(pages)

In [11]:
docs

[Document(metadata={'source': '/kaggle/input/fhcbahbi/Climate Change.en.gu.pdf', 'page': 0, 'page_label': '1'}, page_content='ક્લાઈમેટ ચેન્જ\nઆબોહવા પિરવર્તન એ આપણા સમયના સૌથી મહત્વપૂર્ણ અને દબાણયુક્ત મુદ્દાઓ પૈકી \nએક છે, જે આપણા જીવનના દરેક પાસાઓને અસર કરે છે, પર્યાવરણથી લઈને અર્થતંત્ર, \nઆરોગ્ય અને સુરક્ષા સુધી. તે તાપમાનના લાંબા ગાળાના ફેરફાર અને સ્થાનના લાક્ષિણક \nહવામાન પેટર્નનો સંદર્ભ આપે છે. જ્યારે આબોહવા પિરવર્તન એ કુદરતી ઘટના છે, ત્યારે \nમાનવીય પ્રવૃત્િતઓ, ખાસ કરીને ઔદ્યોિગક ક્રાંિત પછીથી, તેની ગિતમાં નોંધપાત્ર વધારો \nથયો છે, જે પૃથ્વીની આબોહવા પ્રણાલીમાં અભૂતપૂર્વ અને સંભિવત િવનાશક ફેરફારો તરફ \nદોરી જાય છે.\nઆબોહવા પિરવર્તન પર વૈજ્ઞાિનક સર્વસંમિત સ્પષ્ટ છે: પૃથ્વીની આબોહવા ગરમ થઈ રહી \nછે, અને માનવ પ્રવૃત્િતઓ પ્રાથિમક ડ્રાઈવર છે. આબોહવા િવજ્ઞાન પર અગ્રણી ઓથોિરટી \nઓન ક્લાઈમેટ ચેન્જ (IPCC) પરની આંતર સરકારી પેનલે અહેવાલ આપ્યો છે કે 19મી સદીના \nઅંતથી વૈશ્િવક તાપમાનમાં આશરે 1.2 િડગ્રી સેલ્િસયસ (2.2 િડગ્રી ફેરનહીટ) વધારો થયો છે. \nઆ વોર્િમંગ વલણ મોટે ભાગે વાતાવરણમાં ગ્રીનહાઉસ

In [12]:
embeddings = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")
vectorstore = DocArrayInMemorySearch.from_documents(docs, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

INFO - docarray - DB config created
INFO - docarray - Runtime config created
INFO - docarray - No docs or index file provided. Initializing empty InMemoryExactNNIndex.


In [13]:
template = """નીચે આપેલા સંદર્ભના આધારે પ્રશ્નનો જવાબ આપો.
મહત્વપૂર્ણ માહિતી અને ઉદાહરણો સાથે વિગતવાર અને સરળ ભાષામાં સમજાવતો જવાબ આપો.
જો તમારે ખબર ન હોય, તો કહો "મને ખબર નથી".

📌 **સંદર્ભ:** {context}

❓ **પ્રશ્ન:** {question}

➡️ **વિગતવાર જવાબ:**"""

In [14]:
prompt = PromptTemplate.from_template(template)

In [15]:
model_name = "ai4bharat/indicbart"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to("cuda" if torch.cuda.is_available() else "cpu")

In [16]:
def generate_answer(question, context):
    if not context.strip():
        return "માફ કરશો, હું સંદર્ભ શોધી શકતો નથી."

    input_text = f"પ્રશ્ન: {question}\n\nસંદર્ભ: {context}"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=512)

    # Move inputs to correct device and remove 'token_type_ids' key if present
    inputs = {key: value.to("cuda" if torch.cuda.is_available() else "cpu") for key, value in inputs.items() if key != "token_type_ids"}

    # Generate response
    output = model.generate(**inputs, max_length=300)
    answer = tokenizer.decode(output[0], skip_special_tokens=True)

    return answer

In [17]:
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

In [18]:


# ✅ Start Interactive Q&A Loop
print("🔥 **Gujarati Q&A System (Type 'exit' to quit)** 🔥\n")

while True:
    # Get user input
    question = input("You: ")
    
    if question.lower().strip() == 'exit':
        print("👋 બહાર નીકળે છે... સુખદ દિવસ! 🙌")
        break
        
    if not question.strip():
        print("⚠️ કૃપા કરીને યોગ્ય પ્રશ્ન દાખલ કરો.\n")
        continue
    
    # Retrieve relevant context
    retrieved_docs = retriever.get_relevant_documents(question)
    context = format_docs(retrieved_docs) if retrieved_docs else "માફ કરશો, હું સંદર્ભ શોધી શકતો નથી."

    # Generate answer
    answer = generate_answer(question, context)
    
    print("\n🤖 **Bot:**")
    print(f"📜 **Answer:**\n{answer}\n")
    
    # Display sources
    print("📚 **Sources:**")
    for doc in retrieved_docs[:3]:  
        print(f"- {doc.metadata['source']} (Page {doc.metadata['page']+1})") 
    
    print("\n" + "—" * 60 + "\n")

🔥 **Gujarati Q&A System (Type 'exit' to quit)** 🔥



You:  જલવાયુ પરિવર્તન શાને કહેવાય?



🤖 **Bot:**
📜 **Answer:**
મ પ્રશ્ન: જલવાયુ પરિવર્તન શાને કમેવાય? સંમર્ટ: ક્લાટમેટ સેન્જ આમોોોોોોોોો પિરવર્તન એ આપોા સમયના સઅટી મમત્વકો્ટ મોોોોો મોો્મોોોો મુમ્ત મુમ્ોામ એક સે કોોોોો જીવનના મુમ્ત મુમ્ોામ સી સકી એક સે, જે આસોા જીવનના સ્લાસામને મરે મરે મરે મેટ કરે મર્ો સમોોમરોોમરોોી લોમે કર્ો્ર્ટ, મરોોા સમોોોોો સમોો સઅટો. તે [CLS]

📚 **Sources:**
- /kaggle/input/fhcbahbi/Climate Change.en.gu.pdf (Page 1)
- /kaggle/input/fhcbahbi/Climate Change.en.gu.pdf (Page 2)
- /kaggle/input/fhcbahbi/Climate Change.en.gu.pdf (Page 2)

————————————————————————————————————————————————————————————



You:  exit


👋 બહાર નીકળે છે... સુખદ દિવસ! 🙌


In [ ]:
# # Install required packages (uncomment if running for the first time)
# !pip install -q langchain huggingface_hub transformers torch sentencepiece pypdf docarray

# import os
# from langchain.document_loaders import PyPDFLoader
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.embeddings import HuggingFaceEmbeddings
# from langchain.vectorstores import DocArrayInMemorySearch
# from langchain.chains import RetrievalQA
# from langchain.llms import HuggingFaceHub
# from langchain.prompts import PromptTemplate

# # ✅ Load PDF (Update path if needed)
# loader = PyPDFLoader("/kaggle/input/egjbbjb/Climate Change.pdf")  # Update with actual path
# pages = loader.load()

# # ✅ Split documents for better retrieval
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# docs = text_splitter.split_documents(pages)

# # ✅ Create embeddings using Hugging Face model
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

# # ✅ Store documents in a vector database
# vectorstore = DocArrayInMemorySearch.from_documents(docs, embeddings)

# # ✅ Retrieve more relevant documents (k=5 for richer context)
# retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# # ✅ Improved Prompt Template for More Detailed Responses
# template = """Answer the question based on the context below. 
# Provide a **detailed and well-explained** response. If you are unsure, say "I don't know".

# Context: {context}

# Question: {question}

# Provide a **thorough** answer with examples, statistics, and insights:

# Answer:"""
# prompt = PromptTemplate.from_template(template)

# # ✅ Set Hugging Face API Token (Get from https://huggingface.co/settings/tokens)
# os.environ["HUGGINGFACEHUB_API_TOKEN"] = "xxxxxxxxxxxxxxxxxxxxxxx"  # Replace with your token

# # ✅ Choose a more powerful LLM for longer responses
# llm = HuggingFaceHub(
#     repo_id="google/flan-t5-large",  # Upgrade to a larger model (try flan-t5-xxl or Llama-2)
#     model_kwargs={"temperature": 0.2, "max_length": 1500}  # Increased max_length for detailed responses
# )

# # ✅ Function to Format Retrieved Documents (Concatenation)
# def format_docs(docs):
#     return "\n\n".join([doc.page_content for doc in docs])  # Combine retrieved chunks

# # ✅ Build RAG-Based RetrievalQA Chain
# qa_chain = RetrievalQA.from_chain_type(
#     llm=llm,
#     chain_type="stuff",
#     retriever=retriever,
#     chain_type_kwargs={"prompt": prompt},
#     return_source_documents=True
# )

# # ✅ Interactive Q&A Loop
# print("🔥 Climate Change Q&A System (Type 'exit' to quit) 🔥\n")

# while True:
#     # Get user input
#     question = input("You: ")
    
#     if question.lower().strip() == 'exit':
#         print("👋 Exiting... Have a great day!")
#         break
        
#     if not question.strip():
#         print("⚠️ Please enter a valid question.\n")
#         continue
    
#     # Process question
#     result = qa_chain({"query": question})
    
#     # Display results
#     print("\n🤖 Bot:")
#     print(f"📜 **Answer:**\n{result['result']}\n")
    
#     # ✅ Show multiple sources for better transparency
#     print("📚 **Sources:**")
#     for doc in result['source_documents'][:3]: 
#         print(f"- {doc.metadata['source']} (Page {doc.metadata['page']+1})")
    
#     print("\n" + "—" * 60 + "\n")

In [ ]:
# # Install required packages
# !pip install -q langchain huggingface_hub transformers torch sentencepiece pypdf docarray

# from langchain.document_loaders import PyPDFLoader
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.embeddings import HuggingFaceEmbeddings
# from langchain.vectorstores import DocArrayInMemorySearch
# from langchain.chains import RetrievalQA
# from langchain.llms import HuggingFaceHub
# from langchain.prompts import PromptTemplate

# # Load PDF (Upload your PDF to Kaggle first)
# # Click "+ Add data" button on right panel and upload your PDF
# loader = PyPDFLoader("/kaggle/input/egjbbjb/Climate Change.pdf")  # Update path
# pages = loader.load()

# # Split documents
# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=1000,
#     chunk_overlap=200
# )
# docs = text_splitter.split_documents(pages)

# # Create embeddings
# embeddings = HuggingFaceEmbeddings(
#     model_name="sentence-transformers/all-mpnet-base-v2"
# )

# # Create vector store
# vectorstore = DocArrayInMemorySearch.from_documents(
#     docs,
#     embeddings
# )

# # Create retriever
# retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# template = """Answer the question in a detailed and comprehensive manner based on the context below. 
# If you can't answer, say "I don't know".

# Context: {context}

# Question: {question}

# Provide a detailed answer with explanations, examples, and relevant information:

# Answer:"""


# # Initialize model (using free Hugging Face Hub inference API)
# # Create free token at https://huggingface.co/settings/tokens
# import os
# os.environ["HUGGINGFACEHUB_API_TOKEN"] = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXx"  # Replace with your token

# llm = HuggingFaceHub(
#     repo_id="google/flan-t5-large",
#     model_kwargs={"temperature": 0.2, "max_length": 1500}
# )

# # Create QA chain
# qa_chain = RetrievalQA.from_chain_type(
#     llm=llm,
#     chain_type="stuff",
#     retriever=retriever,
#     chain_type_kwargs={"prompt": prompt},
#     return_source_documents=True
# )

# # Test the system
# questions = [
#     "What is the greenhouse effect?",
#     "What is the primary driver of recent climate change?",
#     "How much has sea level risen since 1900?"
# ]

# for question in questions:
#     print(f"Question: {question}")
#     result = qa_chain({"query": question})
#     print(f"Answer: {result['result']}\n")
#     print("Sources:")
#     for doc in result['source_documents'][:1]:  # Show top 1 source
#         print(f"- {doc.metadata['source']} (page {doc.metadata['page']})\n")
#     print("---"*20)

In [ ]:
# # ... (keep all previous code until the QA chain setup)

# # Interactive Q&A loop
# print("Climate Change Q&A System (type 'exit' to quit)\n")

# while True:
#     # Get user input
#     question = input("You: ")
    
#     if question.lower().strip() == 'exit':
#         print("Exiting...")
#         break
        
#     if not question.strip():
#         print("Please enter a valid question.\n")
#         continue
    
#     # Process question
#     result = qa_chain({"query": question})
    
#     # Display results
#     print("\nBot:")
#     print(f"Answer: {result['result']}")
#     print("\nSources:")
#     for doc in result['source_documents'][:1]:  # Show top 1 source
#         print(f"- {doc.metadata['source']} (page {doc.metadata['page']+1})")  # +1 for human-readable page numbers
#     print("\n" + "---"*20 + "\n")